In [72]:
import numpy as np
from numpy.linalg import norm

In [57]:
class matrix_factorization():
    def __init__(self, data, number_of_features):
        self.data = data
        self.number_of_features = number_of_features        
        self.user_count = data.shape[0]
        self.item_count = data.shape[1]
        self.user_features = np.random.uniform(low=0.1, high=0.9, size=(self.user_count, self.number_of_features))
        self.item_features = np.random.uniform(low=0.1, high=0.9, size=(self.number_of_features, self.item_count))
    
    def mse(self):
        matrix_product = np.matmul(self.user_features, self.item_features)
        return np.sum((self.data - matrix_product)**2)
    
    def single_gradient(self, user_row, item_col, wrt_user_idx=None, wrt_item_idx=None):        
        
        if wrt_user_idx != None and wrt_item_idx != None:            
            return "To many elements!"
        elif wrt_user_idx == None and wrt_item_idx == None:            
            return "Insufficient elements!"
        
        else:
            u_row = self.user_features[user_row, :]
            i_col = self.item_features[:, item_col]
            reference_values = float(self.data[user_row, item_col])
            prediction = float(np.dot(u_row, i_col))
        
            if wrt_user_idx != None:
                row_element = float(i_col[wrt_user_idx])
                gradient = 2 * (reference_values - prediction) * row_element
            else:
                col_element = float(u_row[wrt_item_idx])
                gradient = 2 * (reference_values - prediction) * col_element               
            return gradient
    
    def user_feature_gradient(self, user_row, wrt_user_idx):
        summation = 0
        for col in range(0, self.item_count):
            summation += self.single_gradient(user_row=user_row, item_col=col, wrt_user_idx=wrt_user_idx)
        return summation / self.item_count
    
    def item_feature_gradient(self, item_col, wrt_item_idx):
        summation = 0
        for row in range(0, self.user_count):            
            summation += self.single_gradient(user_row=row, item_col=item_col, wrt_item_idx=wrt_item_idx)            
        return summation / self.user_count
    
    def update_user_features(self, learning_rate):
        for i in range(0, self.user_count):
            for j in range(self.number_of_features):
                self.user_features[i, j] += learning_rate * self.user_feature_gradient(user_row=i, wrt_user_idx=j)
    
    def update_item_features(self, learning_rate):
        for i in range(0, self.number_of_features):
            for j in range(0, self.item_count):                
                self.item_features[i, j] += learning_rate * self.item_feature_gradient(item_col=j, wrt_item_idx=i)
                
    def train(self, learning_rate=0.1, iterations=1000):
        for i in range(iterations):
            self.update_user_features(learning_rate)
            self.update_item_features(learning_rate)
            if (i % 50 == 0):
                print(self.mse())
        
    

## Treinando o modelo

In [116]:
data = np.array([[5,3,1], [1,3,0], [5,4,1], [0, 3, 0], [5, 4, 0]])
data

array([[5, 3, 1],
       [1, 3, 0],
       [5, 4, 1],
       [0, 3, 0],
       [5, 4, 0]])

In [108]:
model = matrix_factorization(data, 5)
model.train()

28.915733675931094
0.16908213906315223
0.0017117215043079468
1.2166438694761487e-05
1.6996242958271694e-07
3.065555586706764e-09
6.607491970745466e-11
1.6042134454072607e-12
4.1579257296107965e-14
1.112339788118202e-15
3.019116166456445e-17
8.247990027585096e-19
2.2599494240525964e-20
6.200723004384611e-22
1.7023316570571086e-23
4.677923010981538e-25
1.2796041822462372e-26
3.590152873337034e-28
1.7921474558087106e-29
9.033050096378819e-30


## Verificando as features

In [118]:
model.user_features

array([[ 0.43362315,  0.59112795,  1.77635166,  0.66199543,  0.78540458],
       [ 0.7665225 ,  0.54715589, -0.51519345,  0.97991785,  0.83787965],
       [ 0.80332458,  0.69134973,  1.3408136 ,  0.57429211,  1.37434562],
       [-0.00473894,  0.37691901, -0.35373212,  0.56288186,  0.27582299],
       [ 1.6045746 ,  1.49142372,  0.99187913,  1.03653392,  0.58719001]])

In [119]:
model.item_features

array([[ 0.75605568,  1.17419961, -0.35232492],
       [ 0.7390992 ,  0.86157942, -0.20687264],
       [ 1.90089253,  0.29548949,  0.41818088],
       [ 0.29019687,  0.82866775,  0.11294725],
       [ 0.84859697,  1.15617948,  0.58244831]])

## Reconstruíndo a matriz original

In [120]:
np.dot(model.user_features, model.item_features)

array([[ 5.00000000e+00,  3.00000000e+00,  1.00000000e+00],
       [ 1.00000000e+00,  3.00000000e+00,  6.18200638e-16],
       [ 5.00000000e+00,  4.00000000e+00,  1.00000000e+00],
       [ 3.15802679e-16,  1.00000000e+00, -4.62410698e-16],
       [ 5.00000000e+00,  5.00000000e+00,  1.32523700e-16]])

## Computando similaridade

In [121]:
def cosine_similarity(A,B):
    return np.dot(A,B)/(norm(A)*norm(B))

## Usuários

In [122]:
cosine_similarity(model.user_features[0, :], model.user_features[2, :])

0.9308499901812838

## Itens

In [ ]:
cosine_similarity(model.user_features[0, :], model.user_features[3, :])